# Генерация праздничных тостов 
Для работы с GPT скачаем предобученную модель.
Мы будем работать с библиотекой `transformers` от Hugging Face и с русскоязычной моделью ruGPT3 от Сбера.

In [ ]:
!pip install transformers 

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 26.1 MB/s 
     |████████████████████████████████| 163 kB 64.4 MB/s 
     |████████████████████████████████| 7.6 MB 59.0 MB/s 


Downloading:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/551M [00:00<?, ?B/s]

## Обучающие данные
Будем учить GPT генерировать праздничные тосты. Спарсим сайт https://pozdravok.com/toast/ и выгрузим полученные данные в dataset.txt

In [ ]:
f=open("dataset.txt", 'r')
text = f.read()

In [ ]:
import re

result = len(re.findall(r'\w+', text))
result1 = len(re.findall(r'\w', text))

print("There are " + str(result) + " words.")
print("There are " + str(result1) + " chars.")

There are 52656 words.
There are 278994 chars.


In [ ]:
from functools import reduce
reduce( lambda d, c: d.update([(c, d.get(c,0)+1)]) or d, text.split(), {}) 

{'От': 37,
 'всей': 61,
 'души': 58,
 'поздравляю': 42,
 'с': 768,
 'праздником!': 52,
 'Желаю': 479,
 'безмерного': 4,
 'счастья,': 114,
 'удачи,': 37,
 'непоколебимого': 2,
 'здоровья,': 116,
 'побольше': 52,
 'белых': 1,
 'полос': 4,
 'по': 127,
 'жизни': 197,
 'и': 3253,
 'самых': 49,
 'добрых,': 2,
 'искренних': 10,
 'чудесных': 10,
 'моментов!': 5,
 'В': 137,
 'этот': 240,
 'прекрасный': 28,
 'день': 262,
 'хочется': 42,
 'пожелать': 140,
 'успехов,': 23,
 'благополучия,': 30,
 'творческих': 5,
 'побед': 10,
 'увлекательных': 4,
 'приключений.': 3,
 'Пусть': 895,
 'все,': 21,
 'что': 272,
 'вы': 41,
 'наметили': 1,
 'сбудется': 8,
 'как': 236,
 'можно': 51,
 'скорее.': 2,
 'Жизнь': 12,
 'прекрасна': 2,
 '—': 311,
 'наслаждайтесь': 2,
 'каждым': 29,
 'ее': 22,
 'мигом!': 1,
 'Поздравляю': 270,
 'желаю,': 40,
 'чтобы': 549,
 'жизнь': 211,
 'протекала': 1,
 'без': 67,
 'проблем,': 6,
 'зарядом': 1,
 'позитива,': 15,
 'добрыми': 10,
 'людьми': 6,
 'рядом': 61,
 'невероятным': 3,
 'ус

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

train_path = 'dataset.txt'

train_dataset = TextDataset(tokenizer=tokenizer,file_path=train_path,block_size=64)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


## Training
Для файнтюнинга нам понадобится объект класса Trainer. Далее нужно будет всего-навсего запустить `trainer.train()`

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuned", 
    overwrite_output_dir=True,
    num_train_epochs=200,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=10,
    gradient_accumulation_steps=16,
    fp16=True
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None)
)

Using cuda_amp half precision backend


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 1202
  Num Epochs = 200
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 16
  Total optimization steps = 400


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=400, training_loss=2.7697552490234374, metrics={'train_runtime': 1453.551, 'train_samples_per_second': 165.388, 'train_steps_per_second': 0.275, 'total_flos': 7846011813888000.0, 'train_loss': 2.7697552490234374, 'epoch': 199.84})

# Результат файнтюнинга
Будем использовать сэмплирование с ограничением

In [ ]:
text = "С Новым Годом!\n"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()

with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5,
                        temperature=1.8,
                        top_p=0.8,
                        top_k=50,
                        max_length=80,
                        min_length=30,
                        no_repeat_ngram_size=2, 
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



С Новым Годом!

Новый Год — это время, когда мы становимся взрослее и мудрее. Желаю вам в этом году быть такими же счастливыми и беззаботными, как и раньше. Пусть в вашем доме царит мир, покой и гармония, пусть ваши глаза сияют ярче, чем в тусклом и тёмном зимнем небе.
Поздравляю Вас с Новым годом!
